In [1]:
# Importing packages
import time, random
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [2]:
# SQL setup
from sqlalchemy import create_engine
# connect to a local database
engine = create_engine("sqlite:///pgastats.db")

In [1]:
#Loading in the links to be scraped

links=[]
# open file and read the content in a list
with open(r'link_lists/eoff.txt', 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]

        # add current item to the list
        links.append(x)

In [4]:
# Import saved pickle dataframe from last save point
# Or start with empty dataframe from scratch


main_df=pd.read_pickle('saved_main_df.pkl')
prev_stat_name=''

In [5]:


for link in links[36329:]:
    print('Link: '+link)
    
    # Request HTML of the page and soup it
    url=link
    headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}
    time.sleep(1+2*random.random())
    response=requests.get(url,headers=headers)
    time.sleep(1+2*random.random())
    print('Status code: '+str(response.status_code))
    html=response.text
    soup=bs(html)
    
    # Grab the statistic name
    stat_name=soup.find('div',class_='breadcrumbs').find('a',class_='current').text
    print('Stat: '+stat_name)
    
    # initialize the first previous stat name
    if prev_stat_name == '':
        prev_stat_name=stat_name
    
    
    # Check for dataframe reset
    if stat_name == prev_stat_name:
        pass
    else:
        # Save the dataframe 
        main_df.to_sql('%s'%(prev_stat_name),engine,index=False)
        # reset dataframe to empty dataframe
        main_df=pd.DataFrame()
        print('Table saved to database: '+prev_stat_name)
    
    # Grab the year
    year=soup.find('div',class_='statistics-details-select-wrap season').find('option',selected='selected').text
    print('Year: '+year)
    
    # Grab the time period
    time_period=soup.find('div',class_='statistics-details-select-wrap period').find('option',selected='selected').text
    print('Time period: '+time_period)
    
    # Grab the tournament name
    tournament_name=soup.find('div',class_='statistics-details-select-wrap tournament').find('option',selected='selected').text
    print('Tournament: '+tournament_name)
    
    # Grabs HTML of entire table
    table=soup.find('div',class_='details-table-wrap')
    
    # Grabs just the header of the table (columns)
    table_head=table.find('thead')
    
    # Grabs the column name indiviual tags
    columns=table_head.find_all('th')
    
    # Loop through column tags and save the column names to list
    column_names=[]
    for tag in columns:
        column_names.append(tag.text.strip())
        
    # Grabs the body of the table (rows)
    table_body=table.find('tbody')
    
    # Grabs the row indiviual tags
    rows=table_body.find_all('tr')
    
    # Build a list of lists that contains the values from each row
    row_list=[]
    for tag in rows:
        values=tag.find_all('td')
        value_list=[]
        for value_tag in values:
            value_list.append(value_tag.text.strip())
        row_list.append(value_list)
        
    # Make list of dictionaries that will be used to make dataframe
    # Each dictionary in the list is a row
    row_dict_list=[]
    for row in row_list:
        dictionary=dict(zip(column_names,row))
        row_dict_list.append(dictionary)
    
    # Make dataframe
    df=pd.DataFrame(row_dict_list)
    
    # Add the year, time period, and tournament columns to the dataframe
    df['Year']=year
    df['Time Period']=time_period
    df['Tournament']=tournament_name
    
    # Add dataframe to end of the main one
    main_df=pd.concat([main_df,df])
    
    # Save new previous stat name as current one
    prev_stat_name=stat_name
    
    # Pickle the dataframe for backup
    saved_main_df=main_df.to_pickle('saved_main_df.pkl')
    print('')
    
# Save the last dataframe 
main_df.to_sql('%s'%(prev_stat_name),engine,index=False)
print('Table saved to database: '+prev_stat_name)


Link: https://www.pgatour.com/stats/stat.014.y2021.eoff.t002.html
Status code: 200
Stat: Career Earnings
Year: 2021
Time period: Year-To-Date through
Tournament: The American Express

Link: https://www.pgatour.com/stats/stat.014.y2021.eoff.t006.html
Status code: 200
Stat: Career Earnings
Year: 2021
Time period: Year-To-Date through
Tournament: Sony Open in Hawaii

Link: https://www.pgatour.com/stats/stat.014.y2021.eoff.t016.html
Status code: 200
Stat: Career Earnings
Year: 2021
Time period: Year-To-Date through
Tournament: Sentry Tournament of Champions

Link: https://www.pgatour.com/stats/stat.014.y2021.eoff.t457.html
Status code: 200
Stat: Career Earnings
Year: 2021
Time period: Year-To-Date through
Tournament: Mayakoba Golf Classic presented by UNIFIN

Link: https://www.pgatour.com/stats/stat.014.y2021.eoff.t493.html
Status code: 200
Stat: Career Earnings
Year: 2021
Time period: Year-To-Date through
Tournament: The RSM Classic

Link: https://www.pgatour.com/stats/stat.014.y2021.eoff

Stat: Career Earnings
Year: 2020
Time period: Year-To-Date through
Tournament: Bermuda Championship

Link: https://www.pgatour.com/stats/stat.014.y2020.eoff.t527.html
Status code: 200
Stat: Career Earnings
Year: 2020
Time period: Year-To-Date through
Tournament: ZOZO CHAMPIONSHIP

Link: https://www.pgatour.com/stats/stat.014.y2020.eoff.t521.html
Status code: 200
Stat: Career Earnings
Year: 2020
Time period: Year-To-Date through
Tournament: THE CJ CUP @ NINE BRIDGES

Link: https://www.pgatour.com/stats/stat.014.y2020.eoff.t020.html
Status code: 200
Stat: Career Earnings
Year: 2020
Time period: Year-To-Date through
Tournament: Houston Open

Link: https://www.pgatour.com/stats/stat.014.y2020.eoff.t047.html
Status code: 200
Stat: Career Earnings
Year: 2020
Time period: Year-To-Date through
Tournament: Shriners Hospitals for Children Open

Link: https://www.pgatour.com/stats/stat.014.y2020.eoff.t464.html
Status code: 200
Stat: Career Earnings
Year: 2020
Time period: Year-To-Date through
Tou

Status code: 200
Stat: Career Earnings
Year: 2019
Time period: Year-To-Date through
Tournament: Sony Open in Hawaii

Link: https://www.pgatour.com/stats/stat.014.y2019.eoff.t016.html
Status code: 200
Stat: Career Earnings
Year: 2019
Time period: Year-To-Date through
Tournament: Sentry Tournament of Champions

Link: https://www.pgatour.com/stats/stat.014.y2019.eoff.t493.html
Status code: 200
Stat: Career Earnings
Year: 2019
Time period: Year-To-Date through
Tournament: The RSM Classic

Link: https://www.pgatour.com/stats/stat.014.y2019.eoff.t457.html
Status code: 200
Stat: Career Earnings
Year: 2019
Time period: Year-To-Date through
Tournament: Mayakoba Golf Classic

Link: https://www.pgatour.com/stats/stat.014.y2019.eoff.t047.html
Status code: 200
Stat: Career Earnings
Year: 2019
Time period: Year-To-Date through
Tournament: Shriners Hospitals for Children Open

Link: https://www.pgatour.com/stats/stat.014.y2019.eoff.t489.html
Status code: 200
Stat: Career Earnings
Year: 2019
Time peri

Status code: 200
Stat: Career Earnings
Year: 2018
Time period: Year-To-Date through
Tournament: AT&T Pebble Beach Pro-Am

Link: https://www.pgatour.com/stats/stat.014.y2018.eoff.t003.html
Status code: 200
Stat: Career Earnings
Year: 2018
Time period: Year-To-Date through
Tournament: Waste Management Phoenix Open

Link: https://www.pgatour.com/stats/stat.014.y2018.eoff.t004.html
Status code: 200
Stat: Career Earnings
Year: 2018
Time period: Year-To-Date through
Tournament: Farmers Insurance Open

Link: https://www.pgatour.com/stats/stat.014.y2018.eoff.t002.html
Status code: 200
Stat: Career Earnings
Year: 2018
Time period: Year-To-Date through
Tournament: CareerBuilder Challenge

Link: https://www.pgatour.com/stats/stat.014.y2018.eoff.t006.html
Status code: 200
Stat: Career Earnings
Year: 2018
Time period: Year-To-Date through
Tournament: Sony Open in Hawaii

Link: https://www.pgatour.com/stats/stat.014.y2018.eoff.t016.html
Status code: 200
Stat: Career Earnings
Year: 2018
Time period: 

Status code: 200
Stat: Career Earnings
Year: 2017
Time period: Year-To-Date through
Tournament: Valspar Championship

Link: https://www.pgatour.com/stats/stat.014.y2017.eoff.t473.html
Status code: 200
Stat: Career Earnings
Year: 2017
Time period: Year-To-Date through
Tournament: World Golf Championships-Mexico Championship

Link: https://www.pgatour.com/stats/stat.014.y2017.eoff.t010.html
Status code: 200
Stat: Career Earnings
Year: 2017
Time period: Year-To-Date through
Tournament: The Honda Classic

Link: https://www.pgatour.com/stats/stat.014.y2017.eoff.t007.html
Status code: 200
Stat: Career Earnings
Year: 2017
Time period: Year-To-Date through
Tournament: Genesis Open

Link: https://www.pgatour.com/stats/stat.014.y2017.eoff.t005.html
Status code: 200
Stat: Career Earnings
Year: 2017
Time period: Year-To-Date through
Tournament: AT&T Pebble Beach Pro-Am

Link: https://www.pgatour.com/stats/stat.014.y2017.eoff.t003.html
Status code: 200
Stat: Career Earnings
Year: 2017
Time period: 

...

Status code: 200
Stat: Career Earnings
Year: 2008
Time period: Year-To-Date through
Tournament: World Golf Championships-Bridgestone Invitational

Link: https://www.pgatour.com/stats/stat.014.y2008.eoff.t472.html
Status code: 200
Stat: Career Earnings
Year: 2008
Time period: Year-To-Date through
Tournament: Legends Reno-Tahoe Open

Link: https://www.pgatour.com/stats/stat.014.y2008.eoff.t032.html
Status code: 200
Stat: Career Earnings
Year: 2008
Time period: Year-To-Date through
Tournament: RBC Canadian Open

Link: https://www.pgatour.com/stats/stat.014.y2008.eoff.t100.html
Status code: 200
Stat: Career Earnings
Year: 2008
Time period: Year-To-Date through
Tournament: The Open Championship

Link: https://www.pgatour.com/stats/stat.014.y2008.eoff.t029.html
Status code: 200
Stat: Career Earnings
Year: 2008
Time period: Year-To-Date through
Tournament: U.S. Bank Championship in Milwaukee

Link: https://www.pgatour.com/stats/stat.014.y2008.eoff.t030.html
Status code: 200
Stat: Career Earni

Status code: 200
Stat: Career Earnings
Year: 2007
Time period: Year-To-Date through
Tournament: BMW Championship

Link: https://www.pgatour.com/stats/stat.014.y2007.eoff.t505.html
Status code: 200
Stat: Career Earnings
Year: 2007
Time period: Year-To-Date through
Tournament: Deutsche Bank Championship

Link: https://www.pgatour.com/stats/stat.014.y2007.eoff.t027.html
Status code: 200
Stat: Career Earnings
Year: 2007
Time period: Year-To-Date through
Tournament: The Barclays

Link: https://www.pgatour.com/stats/stat.014.y2007.eoff.t013.html
Status code: 200
Stat: Career Earnings
Year: 2007
Time period: Year-To-Date through
Tournament: Wyndham Championship

Link: https://www.pgatour.com/stats/stat.014.y2007.eoff.t033.html
Status code: 200
Stat: Career Earnings
Year: 2007
Time period: Year-To-Date through
Tournament: PGA Championship

Link: https://www.pgatour.com/stats/stat.014.y2007.eoff.t476.html
Status code: 200
Stat: Career Earnings
Year: 2007
Time period: Year-To-Date through
Tourna

Stat: Career Earnings
Year: 2006
Time period: Year-To-Date through
Tournament: Chrysler Classic of Greensboro

Link: https://www.pgatour.com/stats/stat.014.y2006.eoff.t473.html
Status code: 200
Stat: Career Earnings
Year: 2006
Time period: Year-To-Date through
Tournament: World Golf Championships-American Express Championship

Link: https://www.pgatour.com/stats/stat.014.y2006.eoff.t054.html
Status code: 200
Stat: Career Earnings
Year: 2006
Time period: Year-To-Date through
Tournament: Southern Farm Bureau Classic

Link: https://www.pgatour.com/stats/stat.014.y2006.eoff.t041.html
Status code: 200
Stat: Career Earnings
Year: 2006
Time period: Year-To-Date through
Tournament: Valero Texas Open

Link: https://www.pgatour.com/stats/stat.014.y2006.eoff.t474.html
Status code: 200
Stat: Career Earnings
Year: 2006
Time period: Year-To-Date through
Tournament: 84 LUMBER Classic

Link: https://www.pgatour.com/stats/stat.014.y2006.eoff.t032.html
Status code: 200
Stat: Career Earnings
Year: 2006
T

Status code: 200
Stat: Career Earnings
Year: 2005
Time period: Year-To-Date through
Tournament: THE TOUR Championship presented by Coca-Cola

Link: https://www.pgatour.com/stats/stat.014.y2005.eoff.t054.html
Status code: 200
Stat: Career Earnings
Year: 2005
Time period: Year-To-Date through
Tournament: Southern Farm Bureau Classic

Link: https://www.pgatour.com/stats/stat.014.y2005.eoff.t475.html
Status code: 200
Stat: Career Earnings
Year: 2005
Time period: Year-To-Date through
Tournament: Chrysler Championship

Link: https://www.pgatour.com/stats/stat.014.y2005.eoff.t045.html
Status code: 200
Stat: Career Earnings
Year: 2005
Time period: Year-To-Date through
Tournament: FUNAI Classic at the WALT DISNEY WORLD Resort

Link: https://www.pgatour.com/stats/stat.014.y2005.eoff.t047.html
Status code: 200
Stat: Career Earnings
Year: 2005
Time period: Year-To-Date through
Tournament: Michelin Championship at Las Vegas

Link: https://www.pgatour.com/stats/stat.014.y2005.eoff.t473.html
Status c

Stat: Career Earnings
Year: 2004
Time period: Year-To-Date through
Tournament: World Golf Championships-Accenture Match Play Championship

Link: https://www.pgatour.com/stats/stat.014.y2004.eoff.t001.html
Status code: 200
Stat: Career Earnings
Year: 2004
Time period: Year-To-Date through
Tournament: Chrysler Classic of Tucson

Link: https://www.pgatour.com/stats/stat.014.y2004.eoff.t007.html
Status code: 200
Stat: Career Earnings
Year: 2004
Time period: Year-To-Date through
Tournament: Nissan Open

Link: https://www.pgatour.com/stats/stat.014.y2004.eoff.t004.html
Status code: 200
Stat: Career Earnings
Year: 2004
Time period: Year-To-Date through
Tournament: Buick Invitational

Link: https://www.pgatour.com/stats/stat.014.y2004.eoff.t005.html
Status code: 200
Stat: Career Earnings
Year: 2004
Time period: Year-To-Date through
Tournament: AT&T Pebble Beach National Pro-Am

Link: https://www.pgatour.com/stats/stat.014.y2004.eoff.t003.html
Status code: 200
Stat: Career Earnings
Year: 2004
T

In [6]:
links.index('https://www.pgatour.com/stats/stat.014.y2021.eoff.t002.html')

36329

In [7]:
main_df

,RANK THIS WEEK,RANK LAST WEEK,PLAYER NAME,MONEY,Year,Time Period,Tournament
0,1,1,Tiger Woods,"$120,895,206",2022,Year-To-Date through,BMW Championship
1,2,2,Phil Mickelson,"$94,955,060",2022,Year-To-Date through,BMW Championship
2,3,3,Dustin Johnson,"$74,897,059",2022,Year-To-Date through,BMW Championship
3,4,4,Jim Furyk,"$71,507,269",2022,Year-To-Date through,BMW Championship
4,5,5,Vijay Singh,"$71,236,216",2022,Year-To-Date through,BMW Championship
...,...,...,...,...,...,...,...
706,707,,John Reeves,"$3,563",2004,Year-To-Date through,Mercedes Championships
707,708,,Arjun Atwal,"$3,506",2004,Year-To-Date through,Mercedes Championships
708,709,,Michael Fergin,"$2,835",2004,Year-To-Date through,Mercedes Championships
709,710,,Todd Gleaton,"$2,484",2004,Year-To-Date through,Mercedes Championships


In [2]:
len(links)

37119